<img src="loom.png" align="left" width="40%"/>

AD 1425 [Hausbücher der Nürnberger Zwölfbrüderstiftungen](http://www.nuernberger-hausbuecher.de/75-Amb-2-317-4-v/data)

<img align="right" src="tf-small.png"/>

# Tutorial

This notebook gets you hands-on with
[Text-Fabric](https://github.com/ETCBC/text-fabric) an API on richly annotated text.

Below we show most of its functions in action on the BHSA data set (Hebrew Bible).

In no time you will be a professional weaver.

The tutorial is best understood after having familiarized yourself with the underlying
[data model](https://github.com/ETCBC/text-fabric/wiki/Data-model).

If you want to *get* this all, see the 
[home page](https://github.com/ETCBC/text-fabric/wiki)
of the Text-Fabric wiki.

In [1]:
import sys, os, collections
from tf.fabric import Fabric

# Call Text-Fabric

Everything starts by setting up Text-Fabric.
It needs to know where to look for data.

If you do not specify anything particular, text-fabric looks in some standard locations for a
`text-fabric-data` directory. 

But here we want to illustrate text-fabric with the Hebrew Bible Text Database, a resource with a lot of
high quality data in it.

The data in this resource is so comprehensive, that some of the data is in a core repository,
and some other pieces are in other repositories, as modules.

All this data is versioned together.

For the rest, we assume you have cloned [bhsa](https://github.com/ETCBC/bhsa)
and [phono](https://github.com/ETCBC/phono)
in your directory `~/github/etcbc`.

In [2]:
DATABASE = '~/github/etcbc'
VERSION = '2017'
BHSA = f'bhsa/tf/{VERSION}'
PHONO = f'phono/tf/{VERSION}'
TF = Fabric(locations=[DATABASE], modules=[BHSA, PHONO], silent=False )

This is Text-Fabric 3.1.1
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

118 features found and 0 ignored


Note that we have added a module: `phono`. 
The  core data has a special 1-1 transcription from Hebrew to ASCII, but not a *phonetic* transcription,
taking into account the usual pronunciation rules.

I have made a 
[notebook](https://github.com/ETCBC/phono/blob/master/programs/phono.ipynb)
that tries hard to find phonological representations for all the words.
The notebook has added the result to Text-Fabric as a module.
We'll encounter that later.

**NB:** This is a real-world example of how to add data to an existing datasource as a module.

And if you want to see what those standard locations are, look
[here](https://github.com/Dans-labs/text-fabric/blob/master/tf/fabric.py).

# Load Features
Specify the features to load, and receive the API to work with that data.

In [3]:
api = TF.load('''
    sp lex voc_lex_utf8
    g_word trailer
    qere qere_trailer
    language freq_lex gloss
    mother
''')

  0.00s loading features ...
   |     0.15s B g_word               from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s B qere                 from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s B qere_trailer         from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.08s B trailer              from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.13s B sp                   from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.14s B lex                  from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.01s B voc_lex_utf8         from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.27s B language             from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.13s B freq_lex             from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.01s B gloss                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.23s B mother               from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s Feature overview: 111 for nodes; 5 for edges; 2 configs; 7 comput

Let us see that `loadLog()`.

In [4]:
api.loadLog()

   |     0.00s M otext                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.03s B otype                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.60s B oslots               from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M otext                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M otext@phono          from /Users/dirk/github/etcbc/phono/tf/2017
   |     0.01s B book                 from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.01s B chapter              from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s B verse                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.13s B g_cons               from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.19s B g_cons_utf8          from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.14s B g_lex                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.20s B g_lex_utf8           from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.15s B g_word    

Make it so that the members of the API are directly accessible as global variables.

In [5]:
api.makeAvailableIn(globals())

This is just to show that `F` and `N` are defined and store the same values as `api.F` and `api.N`.

In [6]:
print(F)
print(api.F)
print(N)
print(api.N)

<bound method Api.N of <tf.api.Api object at 0x116828198>>
<bound method Api.N of <tf.api.Api object at 0x116828198>>


# Counting

In order to get acquainted with the data, we start with simple tasks: counting.

## Count all nodes
We use the 
[`N()` generator](https://github.com/ETCBC/text-fabric/wiki/Api#walking-through-nodes)
to walk us through the nodes.

In [7]:
indent(reset=True)
info('Counting nodes ...')
i = 0
for n in N(): i += 1
info('{} nodes'.format(i))

  0.00s Counting nodes ...
  0.38s 1446635 nodes


## Sort some nodes

Get some nodes, 
[slot](https://github.com/ETCBC/text-fabric/wiki/Data-model#summary)
and non-slot, and sort them in the 
[canonical order](https://github.com/ETCBC/text-fabric/wiki/Api#sorting-nodes).

The [`otype` feature](https://github.com/ETCBC/text-fabric/wiki/Data-model#otype-node-feature)
is a
[GRID feature](https://github.com/ETCBC/text-fabric/wiki/Data-model#more-about-the-grid),
a special feature that provides defining characteristics for the
data set as a whole. 
It tells us where the slots end and the other nodes start.

In [8]:
sortNodes(list(range(F.otype.maxSlot+1, F.otype.maxSlot+10))+list(range(1,11)))

[426585,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 426586,
 426587,
 426588,
 426589,
 426590,
 426591,
 426592,
 426593]

## Numbers in the otype feature
Get more information that is readily available in the 
[GRID feature](https://github.com/ETCBC/text-fabric/wiki/Data-model#more-about-the-grid)
[`otype`](https://github.com/ETCBC/text-fabric/wiki/Data-model#otype-node-feature),
namely what types of objects there are in the dataset.

In [9]:
info('{:<9} = {}\n{:<9}={}\n{:<9}={}'.format(
    'slotType', F.otype.slotType,
    'maxSlot', F.otype.maxSlot,
    'maxNode', F.otype.maxNode,
), tm=False)
info('All otypes:\n\t', nl=False, tm=False)
info('\n\t'.join(F.otype.all), tm=False)

slotType  = word
maxSlot  =426584
maxNode  =1446635
All otypes:
	book
	chapter
	lex
	verse
	half_verse
	sentence
	sentence_atom
	clause
	clause_atom
	phrase
	phrase_atom
	subphrase
	word


## Count individual object types

In [10]:
indent(reset=True)
info('counting objects ...')
for otype in F.otype.all:
    i = 0
    indent(level=1, reset=True)
    for n in F.otype.s(otype): i+=1
    info('{:>7} {}s'.format(i, otype))
indent(level=0)
info('Done')

  0.00s counting objects ...
   |     0.00s      39 books
   |     0.00s     929 chapters
   |     0.00s    9233 lexs
   |     0.01s   23213 verses
   |     0.01s   45180 half_verses
   |     0.01s   63711 sentences
   |     0.01s   64486 sentence_atoms
   |     0.01s   88101 clauses
   |     0.02s   90669 clause_atoms
   |     0.04s  253187 phrases
   |     0.04s  267519 phrase_atoms
   |     0.02s  113784 subphrases
   |     0.06s  426584 words
  0.25s Done


# Feature statistics

The content data resides in the features.
The
[`F` function](https://github.com/ETCBC/text-fabric/wiki/Api#node-features)
gives access to that data.
Every feature has a method
[`freqList()`](https://github.com/ETCBC/text-fabric/wiki/Api#node-features)
to generate a frequency list of its values, ordered by highest frequency first.

In [11]:
F.sp.freqList()

(('subs', 125558),
 ('verb', 75450),
 ('prep', 73298),
 ('conj', 62737),
 ('nmpr', 35696),
 ('art', 30387),
 ('adjv', 10075),
 ('nega', 6059),
 ('prps', 5035),
 ('advb', 4603),
 ('prde', 2678),
 ('intj', 1912),
 ('inrg', 1303),
 ('prin', 1026))

# Lexeme matters

## Top 10 frequent verbs

If we count the frequency of words, we usually mean the frequency of their
corresponding lexemes.

There are several methods for working with lexemes.

### Method 1: counting words

In [12]:
verbs = collections.Counter()
indent(reset=True)
info('Collecting data')
for w in F.otype.s('word'):
    if F.sp.v(w) != 'verb': continue
    verbs[F.lex.v(w)] +=1
info('Done')
print(''.join(
    '{}: {}\n'.format(verb, cnt) for (verb, cnt) in sorted(
        verbs.items() , key=lambda x: (-x[1], x[0]))[0:10],
    )
)       

  0.00s Collecting data
  0.33s Done
>MR[: 5378
HJH[: 3561
<FH[: 2629
BW>[: 2570
NTN[: 2017
HLK[: 1554
R>H[: 1298
CM<[: 1168
DBR[: 1138
JCB[: 1082



### Method 2: counting lexemes

An alternative way to do this is to use the feature `freq_lex`, defined for `lex` nodes.
Now we walk the lexemes instead of the occurrences.
Note that the feature `sp` (part-of-speech) is defined for nodes of type `word` as well as `lex`.
Both also have the `lex` feature.
Note that we might encounter Hebrew lexemes as well as Aramaic lexemes, so we still have to
accumulate the `freq_lex`es of the lexeme nodes with the same lexeme value.

In [13]:
verbs = collections.Counter()
indent(reset=True)
info('Collecting data')
for w in F.otype.s('lex'):
    if F.sp.v(w) != 'verb': continue
    verbs[F.lex.v(w)] += F.freq_lex.v(w)
info('Done')
print(''.join(
    '{}: {}\n'.format(verb, cnt) for (verb, cnt) in sorted(
        verbs.items() , key=lambda x: (-x[1], x[0]))[0:10],
    )
)       

  0.00s Collecting data
  0.02s Done
>MR[: 5378
HJH[: 3561
<FH[: 2629
BW>[: 2570
NTN[: 2017
HLK[: 1554
R>H[: 1298
CM<[: 1168
DBR[: 1138
JCB[: 1082



## Lexeme distribution

Let's do a bit more fancy lexeme stuff.

### Hapaxes

A hapax can be found by inspecting lexemes and see to how many word nodes they are linked.
If that is number is one, we have a hapax.

We print 10 hapaxes with their gloss.

In [14]:
hapax = []
zero = set()

indent(reset=True)
for l in F.otype.s('lex'):
    occs = L.d(l, otype='word')
    n = len(occs)
    if n == 0: # that's weird: should not happen
        zero.add(l)
    elif n == 1: # hapax found!
        hapax.append(l)
info('{} hapaxes found'.format(len(hapax)))
if zero:
    error('{} zeroes found'.format(len(zero)), tm=False)
else:
    info('No zeroes found', tm=False)
for h in hapax[0:10]:
    print('\t{:<8} {}'.format(F.lex.v(h), F.gloss.v(h)))

  0.16s 3072 hapaxes found
No zeroes found
	PJCWN/   Pishon
	CWP[     bruise
	HRWN/    pregnancy
	Z<H/     sweat
	LHV/     flame
	NWD/     Nod
	XNWK=/   Enoch
	MXWJ>L/  Mehujael
	MXJJ>L/  Mehujael
	JBL=/    Jabal


### Small occurrence base

The occurrence base of a lexeme are the verses, chapters and books in which occurs.
Let's look for lexemes that occur in a single chapter.

If a lexeme occurs in a single chapter, its slots are a subset of the slots of that chapter.
So, if you go *up* from the lexeme, you encounter the chapter.

Normally, lexemes occur in many chapters, and then none of them totally includes all occurrences of it,
so if you go up from such lexemes, you don not find chapters.

Let's check it out.

Oh yes, we have already found the hapaxes, we will skip them here.

In [15]:
singleCh = []
multiple = []
indent(reset=True)
info('Finding single chapter lexemes')
for l in F.otype.s('lex'):
    chapters = L.u(l, 'chapter')
    if len(chapters) == 1:
        if l not in hapax:
            singleCh.append(l)
    elif len(chapters) > 0: # should not happen
        multipleCh.append(l)
info('{} single chapter lexemes found'.format(len(singleCh)))
if multiple:
    error('{} chapter embedders of multiple lexemes found'.format(len(multiple)), tm=False)
else:
    info('No chapter embedders of multiple lexemes found', tm=False)
for s in singleCh[0:10]:
    print('{:<20} {:<6}'.format(
        '{} {}:{}'.format(*T.sectionFromNode(s)),
        F.lex.v(s),
    ))

  0.00s Finding single chapter lexemes
  0.15s 450 single chapter lexemes found
No chapter embedders of multiple lexemes found
Genesis 4:1          QJN=/ 
Genesis 4:2          HBL=/ 
Genesis 4:18         <JRD/ 
Genesis 4:18         MTWC>L/
Genesis 4:19         YLH/  
Genesis 4:22         TWBL_QJN/
Genesis 10:11        KLX=/ 
Genesis 14:1         >MRPL/
Genesis 14:1         >RJWK/
Genesis 14:1         >LSR/ 


### Confined to books

As a final exercise with lexemes, lets make a list of all books, and show their total number of lexemes and
the number of lexemes that occur exclusively in that book.

In [16]:
indent(reset=True)

allBook = collections.defaultdict(set)
allLex = set()

info('Making book-lexeme index')
for b in F.otype.s('book'):
    for w in L.d(b, 'word'):
        l = L.u(w, 'lex')[0]
        allBook[b].add(l)
        allLex.add(l)
info('Found {} lexemes'.format(len(allLex)))

  0.00s Making book-lexeme index
  4.88s Found 9233 lexemes


In [17]:
indent(reset=True)

singleBook = collections.defaultdict(lambda:0)
info('Finding single book lexemes')
for l in F.otype.s('lex'):
    book = L.u(l, 'book')
    if len(book) == 1:
        singleBook[book[0]] += 1
info('found {} single book lexemes'.format(sum(singleBook.values())))

  0.00s Finding single book lexemes
  0.06s found 4226 single book lexemes


In [18]:
print('{:<20}{:>5}{:>5}{:>5}\n{}'.format(
    'book', '#all', '#own', '%own',
    '-'*35,
))
booklist = []

for b in F.otype.s('book'):
    book = T.bookName(b)
    a = len(allBook[b])
    o = singleBook.get(b, 0)
    p = 100 * o / a
    booklist.append((book, a, o, p))

for x in sorted(booklist, key=lambda e: (-e[3], -e[1], e[0])):
    print('{:<20} {:>4} {:>4} {:>4.1f}%'.format(*x))

book                 #all #own %own
-----------------------------------
Daniel               1121  428 38.2%
1_Chronicles         2015  488 24.2%
Ezra                  991  199 20.1%
Joshua               1175  206 17.5%
Esther                472   67 14.2%
Isaiah               2553  350 13.7%
Numbers              1457  197 13.5%
Ezekiel              1718  212 12.3%
Song_of_songs         503   60 11.9%
Job                  1717  202 11.8%
Genesis              1817  208 11.4%
Nehemiah             1076  110 10.2%
Psalms               2251  216  9.6%
Leviticus             960   89  9.3%
Judges               1210   99  8.2%
Ecclesiastes          575   46  8.0%
Proverbs             1356  103  7.6%
Jeremiah             1949  147  7.5%
2_Samuel             1304   89  6.8%
1_Samuel             1256   85  6.8%
2_Kings              1266   85  6.7%
Exodus               1425   92  6.5%
1_Kings              1291   81  6.3%
Deuteronomy          1449   80  5.5%
Lamentations          592   31  5.2%
2_C

## Part of speech counting
We count the words of each part of speech, and we list to top 10 of frequent lexemes.

**NB**: This is not so much about lexemes as well as
generating pretty progress messages.

In [19]:
partOfSpeech = collections.Counter()
freqLex = collections.Counter()

indent(level=0, reset=True)
info('Starting tasks')
indent(level=1, reset=True)
info('Counting the words by part-of-speech ...')
for w in F.otype.s('word'):
    partOfSpeech[F.sp.v(w)] += 1
info('Done: {} categories'.format(len(partOfSpeech)))
indent(level=2)
info('\n'.join('{:<7}: {:>6}x'.format(*x) for x in sorted(
    partOfSpeech.items(),
    key=lambda x: (-x[1], x[0])
)), tm=False)
indent(level=1, reset=True)
info('Listing the top 10 frequent words ...')
for w in F.otype.s('word'):
    freqLex[F.lex.v(w)] += 1
info('Done: {} lexemes'.format(len(freqLex)))
indent(level=2)
info('\n'.join('{:<7}: {:>6}x'.format(*x) for x in sorted(
    freqLex.items(),
    key=lambda x: (-x[1], x[0])
)[0:10]), tm=False)
indent(level=0)
info('All tasks completed')

  0.00s Starting tasks
   |     0.00s Counting the words by part-of-speech ...
   |     0.43s Done: 14 categories
   |      |   subs   : 121483x
   |      |   verb   :  73710x
   |      |   prep   :  73273x
   |      |   conj   :  62722x
   |      |   nmpr   :  33081x
   |      |   art    :  30386x
   |      |   adjv   :   9464x
   |      |   nega   :   6053x
   |      |   prps   :   5011x
   |      |   advb   :   4550x
   |      |   prde   :   2660x
   |      |   intj   :   1885x
   |      |   inrg   :   1285x
   |      |   prin   :   1021x
   |     0.00s Listing the top 10 frequent words ...
   |     0.42s Done: 8773 lexemes
   |      |   W      :  51003x
   |      |   H      :  30392x
   |      |   L      :  20447x
   |      |   B      :  15768x
   |      |   >T     :  10997x
   |      |   MN     :   7681x
   |      |   JHWH/  :   6828x
   |      |   <L     :   5870x
   |      |   >L     :   5521x
   |      |   >CR    :   5500x
  0.88s All tasks completed


# Layer API
We travel upwards and downwards, forwards and backwards through the nodes.
The Layer-API (`L`) provides functions: `u()` for going up, and `d()` for going down,
`n()` for going to next nodes and `p()` for going to previous nodes.

These directions are indirect notions: nodes are just numbers, but by means of the
`oslots` feature they are linked to slots. One node *contains* an other node, if the one is linked to a set of slots that contains the set of slots that the other is linked to.
And one if next or previous to an other, if its slots follow of precede the slots of the other one.

`L.u(node)` **Up** is going to nodes that embed `node`.

`L.d(node)` **Down** is the opposite direction, to those that are contained in `node`.

`L.n(node)` **Next** are the next *adjacent* nodes, i.e. nodes whose first slot comes immediately after the last slot of `node`.

`L.p(node)` **Previous** are the previous *adjacent* nodes, i.e. nodes whose last slot comes immediately before the first slot of `node`.

All these functions yield nodes of all possible otypes.
By passing an optional parameter, you can restrict the results to nodes of that type.

The result is ordered in the canonical node ordering.
The functions return always a tuple, even if there is just one node in the result.

## Going up
We go from the first word to the book it contains.

In [20]:
firstBook = L.u(1, otype='book')[0]
print(firstBook)

426585


And let's see all the containing objects of word 3:

In [21]:
w = 3
for otype in F.otype.all:
    if otype == F.otype.slotType: continue
    up = L.u(w, otype=otype)
    upNode = 'x' if len(up) == 0 else up[0]
    print('word {} is contained in {} {}'.format(w, otype, upNode))

word 3 is contained in book 426585
word 3 is contained in chapter 426624
word 3 is contained in lex 1437405
word 3 is contained in verse 1414190
word 3 is contained in half_verse 606323
word 3 is contained in sentence 1172209
word 3 is contained in sentence_atom 1235920
word 3 is contained in clause 427553
word 3 is contained in clause_atom 515654
word 3 is contained in phrase 651504
word 3 is contained in phrase_atom 904691
word 3 is contained in subphrase x


## Going next
Let's go to the next nodes of the first book.

In [22]:
afterFirstBook = L.n(firstBook)
for n in afterFirstBook:
    print('{:>7}: {:<13} first slot={:<6}, last slot={:<6}'.format(
        n, F.otype.v(n),
        E.oslots.s(n)[0],
        E.oslots.s(n)[-1],
    ))
secondBook = L.n(firstBook, otype='book')[0]

  28764: word          first slot=28764 , last slot=28764 
 923447: phrase_atom   first slot=28764 , last slot=28764 
 669484: phrase        first slot=28764 , last slot=28764 
 521793: clause_atom   first slot=28764 , last slot=28768 
 433543: clause        first slot=28764 , last slot=28768 
 609323: half_verse    first slot=28764 , last slot=28771 
1240568: sentence_atom first slot=28764 , last slot=28773 
1176828: sentence      first slot=28764 , last slot=28792 
1415723: verse         first slot=28764 , last slot=28777 
 426674: chapter       first slot=28764 , last slot=29112 
 426586: book          first slot=28764 , last slot=52511 


## Going previous

And let's see what is right before the second book.

In [23]:
for n in L.p(secondBook):
    print('{:>7}: {:<13} first slot={:<6}, last slot={:<6}'.format(
        n, F.otype.v(n),
        E.oslots.s(n)[0],
        E.oslots.s(n)[-1],
    ))

 426585: book          first slot=1     , last slot=28763 
 426673: chapter       first slot=28259 , last slot=28763 
1415722: verse         first slot=28746 , last slot=28763 
 609322: half_verse    first slot=28754 , last slot=28763 
1176827: sentence      first slot=28757 , last slot=28763 
1240567: sentence_atom first slot=28757 , last slot=28763 
 433542: clause        first slot=28757 , last slot=28763 
 521792: clause_atom   first slot=28757 , last slot=28763 
 669483: phrase        first slot=28762 , last slot=28763 
 923446: phrase_atom   first slot=28762 , last slot=28763 
  28763: word          first slot=28763 , last slot=28763 


## Going down

We go to the chapters of the second book, and just count them.

In [24]:
chapters = L.d(secondBook, otype='chapter')
print(len(chapters))

40


## The first verse
We pick the first verse and the first word, and explore what is above and below them.

In [25]:
for n in [1, L.u(1, otype='verse')[0]]:
    indent(level=0)
    info('Node {}'.format(n), tm=False)
    indent(level=1)
    info('UP', tm=False)
    indent(level=2)
    info('\n'.join(['{:<15} {}'.format(u, F.otype.v(u)) for u in L.u(n)]), tm=False)
    indent(level=1)
    info('DOWN', tm=False)
    indent(level=2)
    info('\n'.join(['{:<15} {}'.format(u, F.otype.v(u)) for u in L.d(n)]), tm=False)
indent(level=0)
info('Done', tm=False)

Node 1
   |   UP
   |      |   1437403         lex
   |      |   904690          phrase_atom
   |      |   651503          phrase
   |      |   606323          half_verse
   |      |   515654          clause_atom
   |      |   427553          clause
   |      |   1235920         sentence_atom
   |      |   1172209         sentence
   |      |   1414190         verse
   |      |   426624          chapter
   |      |   426585          book
   |   DOWN
   |      |   
Node 1414190
   |   UP
   |      |   426624          chapter
   |      |   426585          book
   |   DOWN
   |      |   1172209         sentence
   |      |   1235920         sentence_atom
   |      |   427553          clause
   |      |   515654          clause_atom
   |      |   606323          half_verse
   |      |   651503          phrase
   |      |   904690          phrase_atom
   |      |   1               word
   |      |   2               word
   |      |   651504          phrase
   |      |   904691          phra

# Text API

We examine the functions of the Text API: `T`.

## Formats
First the formats that we have available to represent the actual text.
These formats have been defined in the `otext` feature.
This is an optional GRID config feature: it has only metadata.

In [26]:
sorted(T.formats)

['lex-orig-full',
 'lex-orig-plain',
 'lex-trans-full',
 'lex-trans-plain',
 'text-orig-full',
 'text-orig-full-ketiv',
 'text-orig-plain',
 'text-phono-full',
 'text-trans-full',
 'text-trans-full-ketiv',
 'text-trans-plain']

Note the `text-phono-full` format here.
It does not come from the main data source `etcbc4c`, but from the module `phono`.
Look in your data directory, find `text-fabric-data/hebrew/phono/otext@phono.tf`,
and you'll see this format defined there.

## Using the formats
Now let's use those formats to print out the first verse of the Hebrew Bible.

In [27]:
for fmt in sorted(T.formats):
    print('{}:\n\t{}'.format(fmt, T.text(range(1,12), fmt=fmt)))

lex-orig-full:
	בְּ רֵאשִׁית בָּרָא אֱלֹה אֵת הַ שָּׁמַי וְ אֵת הָ אָרֶץ 
lex-orig-plain:
	ב ראשׁית ברא אלהים את ה שׁמים ו את ה ארץ 
lex-trans-full:
	B.:- R;>CIJT B.@R@> >:ELOH >;T HA- C.@MAJ W:- >;T H@- >@REY 
lex-trans-plain:
	B R>CJT BR> >LHJM >T H CMJM W >T H >RY 
text-orig-full:
	בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ 
text-orig-full-ketiv:
	בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ 
text-orig-plain:
	בראשׁית ברא אלהים את השׁמים ואת הארץ׃ 
text-phono-full:
	bᵊrēšˌîṯ bārˈā ʔᵉlōhˈîm ʔˌēṯ haššāmˌayim wᵊʔˌēṯ hāʔˈāreṣ . 
text-trans-full:
	B.:-R;>CI73JT B.@R@74> >:ELOHI92JM >;71T HA-C.@MA73JIM W:->;71T H@->@75REY00 
text-trans-full-ketiv:
	B.:-R;>CI73JT B.@R@74> >:ELOHI92JM >;71T HA-C.@MA73JIM W:->;71T H@->@75REY00 
text-trans-plain:
	BR>CJT BR> >LHJM >T HCMJM W>T H>RY00 


If we do not specify a format, the **default** format is used (`text-orig-full`).

In [28]:
print(T.text(range(1,12)))

בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ 


## Whole text in all formats in just 10 seconds
We are going to produce the complete text of the Hebrew Bible in all available formats.

In [29]:
text = collections.defaultdict(list)
indent(reset=True)
info('writing plain text of whole Bible in all formats')
for v in F.otype.s('verse'):
    words = L.d(v, 'word')
    for fmt in sorted(T.formats):
        text[fmt].append(T.text(words, fmt=fmt))
info('done {} formats'.format(len(text)))
for fmt in sorted(text):
    print('{}\n{}\n'.format(fmt, '\n'.join(text[fmt][0:5])))

  0.00s writing plain text of whole Bible in all formats
  9.88s done 11 formats
lex-orig-full
בְּ רֵאשִׁית בָּרָא אֱלֹה אֵת הַ שָּׁמַי וְ אֵת הָ אָרֶץ 
וְ הָ אָרֶץ הָי תֹהוּ וָ בֹהוּ וְ חֹשֶׁךְ עַל פְּן תְהֹום וְ רוּחַ אֱלֹה רַחֶף עַל פְּן הַ מָּי 
וַ אמֶר אֱלֹה הִי אֹור וַ הִי אֹור 
וַ רְא אֱלֹה אֶת הָ אֹור כִּי טֹוב וַ בְדֵּל אֱלֹה בֵּין הָ אֹור וּ בֵין הַ חֹשֶׁךְ 
וַ קְרָא אֱלֹה לָ  אֹור יֹום וְ לַ  חֹשֶׁךְ קָרָא לָיְלָה וַ הִי עֶרֶב וַ הִי בֹקֶר יֹום אֶחָד 

lex-orig-plain
ב ראשׁית ברא אלהים את ה שׁמים ו את ה ארץ 
ו ה ארץ היה תהו ו בהו ו חשׁך על פנה תהום ו רוח אלהים רחף על פנה ה מים 
ו אמר אלהים היה אור ו היה אור 
ו ראה אלהים את ה אור כי טוב ו בדל אלהים בין ה אור ו בין ה חשׁך 
ו קרא אלהים ל ה אור יום ו ל ה חשׁך קרא לילה ו היה ערב ו היה בקר יום אחד 

lex-trans-full
B.:- R;>CIJT B.@R@> >:ELOH >;T HA- C.@MAJ W:- >;T H@- >@REY 
W:- H@- >@REY H@J TOHW. W@- BOHW. W:- XOCEK: <AL P.:N T:HOWM W:- RW.XA >:ELOH RAXEP <AL P.:N HA- M.@J 
WA- >MER >:ELOH HIJ >OWR WA- HIJ >OWR 
WA- R:> >:ELOH >E

### The full plain text
We write the default plain text version to file.

In [30]:
with open(os.path.expanduser('~/Downloads/text-orig-full.txt'), 'w') as f:
    f.write('\n'.join(text['text-orig-full']))

## Book names

For Bible book names, we can use several languages.

### Languages
Here are the languages that we can use for book names.
These languages come from the features `book@ll`, where `ll` is a two letter
ISO language code. Have a look in your data directory, you can't miss them.

In [31]:
T.languages

{'am': {'language': 'ኣማርኛ', 'languageEnglish': 'amharic'},
 'ar': {'language': 'العَرَبِية', 'languageEnglish': 'arabic'},
 'bn': {'language': 'বাংলা', 'languageEnglish': 'bengali'},
 'da': {'language': 'Dansk', 'languageEnglish': 'danish'},
 'de': {'language': 'Deutsch', 'languageEnglish': 'german'},
 'el': {'language': 'Ελληνικά', 'languageEnglish': 'greek'},
 'en': {'language': 'English', 'languageEnglish': 'english'},
 'es': {'language': 'Español', 'languageEnglish': 'spanish'},
 'fa': {'language': 'فارسی', 'languageEnglish': 'farsi'},
 'fr': {'language': 'Français', 'languageEnglish': 'french'},
 'he': {'language': 'עברית', 'languageEnglish': 'hebrew'},
 'hi': {'language': 'हिन्दी', 'languageEnglish': 'hindi'},
 'id': {'language': 'Bahasa Indonesia', 'languageEnglish': 'indonesian'},
 'ja': {'language': '日本語', 'languageEnglish': 'japanese'},
 'ko': {'language': '한국어', 'languageEnglish': 'korean'},
 'la': {'language': 'Latina', 'languageEnglish': 'latin'},
 'nl': {'language': 'Nede

### Book names in Swahili
Get the book names in Swahili.

In [32]:
nodeToSwahili = ''
for b in F.otype.s('book'):
    nodeToSwahili += '{} = {}\n'.format(b, T.bookName(b, lang='sw'))
print(nodeToSwahili)

426585 = Mwanzo
426586 = Kutoka
426587 = Mambo_ya_Walawi
426588 = Hesabu
426589 = Kumbukumbu_la_Torati
426590 = Yoshua
426591 = Waamuzi
426592 = 1_Samweli
426593 = 2_Samweli
426594 = 1_Wafalme
426595 = 2_Wafalme
426596 = Isaya
426597 = Yeremia
426598 = Ezekieli
426599 = Hosea
426600 = Yoeli
426601 = Amosi
426602 = Obadia
426603 = Yona
426604 = Mika
426605 = Nahumu
426606 = Habakuki
426607 = Sefania
426608 = Hagai
426609 = Zekaria
426610 = Malaki
426611 = Zaburi
426612 = Ayubu
426613 = Mithali
426614 = Ruthi
426615 = Wimbo_Ulio_Bora
426616 = Mhubiri
426617 = Maombolezo
426618 = Esta
426619 = Danieli
426620 = Ezra
426621 = Nehemia
426622 = 1_Mambo_ya_Nyakati
426623 = 2_Mambo_ya_Nyakati



## Book nodes from Swahili
OK, there they are. We copy them into a string, and do the opposite: get the nodes back.
We check whether we get exactly the same nodes as the ones we started with.

In [33]:
swahiliNames = '''
Mwanzo
Kutoka
Mambo_ya_Walawi
Hesabu
Kumbukumbu_la_Torati
Yoshua
Waamuzi
1_Samweli
2_Samweli
1_Wafalme
2_Wafalme
Isaya
Yeremia
Ezekieli
Hosea
Yoeli
Amosi
Obadia
Yona
Mika
Nahumu
Habakuki
Sefania
Hagai
Zekaria
Malaki
Zaburi
Ayubu
Mithali
Ruthi
Wimbo_Ulio_Bora
Mhubiri
Maombolezo
Esta
Danieli
Ezra
Nehemia
1_Mambo_ya_Nyakati
2_Mambo_ya_Nyakati
'''.strip().split()

swahiliToNode = ''
for nm in swahiliNames:
    swahiliToNode += '{} = {}\n'.format(T.bookNode(nm, lang='sw'), nm)
    
if swahiliToNode != nodeToSwahili:
    print('Something is not right with the book names')
else:
    print('Going from nodes to booknames and back yields the original nodes')

Going from nodes to booknames and back yields the original nodes


## Sentences spanning multiple verses
If you go up from a sentence node, you expect to find a verse node.
But some sentences span multiple verses, and in that case, you will not find the enclosing
verse node, because it is not there.

Here is a piece of code to detect and list all cases where sentences span multiple verses.

In [34]:
verseSpanners = []
for s in F.otype.s('sentence'):
    if not L.u(s, otype='verse'):
        verseSpanners.append(s)
print(f'{len(verseSpanners)} sentences span multiple verses')
firstSpanner = verseSpanners[0]
print(f'The first one is {firstSpanner}')

892 sentences span multiple verses
The first one is 1172254


## Sections

A section in the Hebrew bible is a book, a chapter or a verse.
Knowledge of sections is not baked into Text-Fabric. 
The config feature `otext.tf` may specify three section levels, and tell
what the corresponding node types and features are.

From that knowledge it can construct mappings from nodes to sections, e.g. from verse
nodes to tuples of the form:

    (bookName, chapterNumber, verseNumber)
   
Here are examples of getting the section that corresponds to a node and vice versa.

**NB:** `sectionFromNode` always delivers a verse specification, either from the
first slot belonging to that node, or, if `lastSlot`, from the last slot
belonging to that node.

In [35]:
allChapters = list(F.otype.s('chapter'))
chapter300 = allChapters[299]

for x in (
    ('section of first word',    T.sectionFromNode(1)                            ),
    ('node of Gen 1:1',          T.nodeFromSection(('Genesis', 1, 1))            ),
    ('idem',                     T.nodeFromSection(('Mwanzo', 1, 1), lang='sw')  ),
    ('node of book Genesis',     T.nodeFromSection(('Genesis',))                 ),
    ('node of Genesis 1',        T.nodeFromSection(('Genesis', 1))               ),
    ('section of sentence node', T.sectionFromNode(firstSpanner)                 ),
    ('idem, now last word',      T.sectionFromNode(firstSpanner, lastSlot=True)  ),
    ('section of chapter node',  T.sectionFromNode(chapter300)                   ),
    ('idem, now last word',      T.sectionFromNode(chapter300, lastSlot=True)    ),
): print('{:<30} {}'.format(*x))

section of first word          ('Genesis', 1, 1)
node of Gen 1:1                1414190
idem                           1414190
node of book Genesis           426585
node of Genesis 1              426624
section of sentence node       ('Genesis', 1, 17)
idem, now last word            ('Genesis', 1, 18)
section of chapter node        ('1_Kings', 13, 1)
idem, now last word            ('1_Kings', 13, 34)


Lets return to the verse spanning sentences and show the first 10 of them.

In [36]:
def showSpanSentence(s):
    words = L.d(s, otype='word')
    firstWord = words[0]
    lastWord = words[-1]
    currentWord = words[0]
    while currentWord <= lastWord:
        currentVerse = L.u(currentWord, otype='verse')[0]
        currentWords = L.d(currentVerse, otype='word')
        currentWordsInSentence = [w for w in currentWords if w <= lastWord]
        currentVerseBefore = currentWords[0] < firstWord
        currentVerseAfter = currentWords[-1] > lastWord
        currentPassage = '{} {}:{}'.format(*T.sectionFromNode(currentVerse))
        print('{} {} "{}" {}'.format(
            currentPassage,
            '...' if currentVerseAfter else '',
            T.text(currentWordsInSentence),
            '...' if currentVerseBefore else '',
        ))
        currentWord = currentWords[-1] + 1
    print('')

for s in verseSpanners[0:10]:
    showSpanSentence(s)

Genesis 1:17  "וַיִּתֵּ֥ן אֹתָ֛ם אֱלֹהִ֖ים בִּרְקִ֣יעַ הַשָּׁמָ֑יִם לְהָאִ֖יר עַל־הָאָֽרֶץ׃ " 
Genesis 1:18 ... "וְלִמְשֹׁל֙ בַּיֹּ֣ום וּבַלַּ֔יְלָה וּֽלֲהַבְדִּ֔יל בֵּ֥ין הָאֹ֖ור וּבֵ֣ין הַחֹ֑שֶׁךְ " 

Genesis 1:29  "וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃ " ...
Genesis 1:30 ... "וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ אֲשֶׁר־בֹּו֙ נֶ֣פֶשׁ חַיָּ֔ה אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה " 

Genesis 2:4  "אֵ֣לֶּה תֹולְדֹ֧ות הַשָּׁמַ֛יִם וְהָאָ֖רֶץ בְּהִבָּֽרְאָ֑ם בְּיֹ֗ום עֲשֹׂ֛ות יְהוָ֥ה אֱלֹהִ֖ים אֶ֥רֶץ וְשָׁמָֽיִם׃ " ...
Genesis 2:5  "וְכֹ֣ל׀ שִׂ֣יחַ הַשָּׂדֶ֗ה טֶ֚רֶם יִֽהְיֶ֣ה בָאָ֔רֶץ וְכָל־עֵ֥שֶׂב הַשָּׂדֶ֖ה טֶ֣רֶם יִצְמָ֑ח כִּי֩ לֹ֨א הִמְטִ֜יר יְהוָ֤ה אֱלֹהִים֙ עַל־הָאָ֔רֶץ וְאָדָ֣ם אַ֔יִן לַֽעֲבֹ֖ד אֶת־הָֽאֲדָמָֽה׃ " 
Genesis 2:6  "וְאֵ֖ד יַֽעֲלֶ֣ה מִן־הָאָ֑רֶץ וְהִשְׁקָ֖ה אֶֽת־כָּל־

# Ketiv Qere
Let us explore where Ketiv/Qere pairs are and how they render.

In [37]:
qeres = [w for w in F.otype.s('word') if F.qere.v(w) != None]
print('{} qeres'.format(len(qeres)))
for w in qeres[0:10]:
    print('{}: ketiv = "{}"+"{}" qere = "{}"+"{}"'.format(
        w, F.g_word.v(w), F.trailer.v(w), F.qere.v(w), F.qere_trailer.v(w),
    ))

1892 qeres
3897: ketiv = "*HWY>"+" " qere = "HAJ:Y;74>"+" "
4420: ketiv = "*>HLH"+" " qere = ">@H:@LO75W"+"00"
5645: ketiv = "*>HLH"+" " qere = ">@H:@LO92W"+" "
5912: ketiv = "*>HLH"+" " qere = ">@95H:@LOW03"+" "
6246: ketiv = "*YBJJM"+" " qere = "Y:BOWJI80m"+" "
6354: ketiv = "*YBJJM"+" " qere = "Y:BOWJI80m"+" "
11761: ketiv = "*W-"+"" qere = "WA"+""
11762: ketiv = "*JJFM"+" " qere = "J.W.FA70m"+" "
12783: ketiv = "*GJJM"+" " qere = "GOWJIm03"+" "
13684: ketiv = "*YJDH"+" " qere = "Y@75JID"+"00"


## Show a ketiv-qere pair
Let us print all text representations of the verse in which word node 4419 occurs.

In [38]:
refWord = qeres[1]
vn = L.u(refWord, otype='verse')[0]
ws = L.d(vn, otype='word')
print('{} {}:{}'.format(*T.sectionFromNode(refWord)))
for fmt in sorted(T.formats):
    if fmt.startswith('text-'):
        print('{:<25} {}'.format(fmt, T.text(ws, fmt=fmt)))

Genesis 9:21
text-orig-full            וַיֵּ֥שְׁתְּ מִן־הַיַּ֖יִן וַיִּשְׁכָּ֑ר וַיִּתְגַּ֖ל בְּתֹ֥וךְ אָהֳלֹֽו׃
text-orig-full-ketiv      וַיֵּ֥שְׁתְּ מִן־הַיַּ֖יִן וַיִּשְׁכָּ֑ר וַיִּתְגַּ֖ל בְּתֹ֥וךְ אהלה 
text-orig-plain           וישׁת מן־היין וישׁכר ויתגל בתוך אהלה 
text-phono-full           wayyˌēšt min-hayyˌayin wayyiškˈār wayyiṯgˌal bᵊṯˌôḵ *ʔohᵒlˈô .
text-trans-full           WA-J.;71C:T.: MIN&HA-J.A73JIN WA-J.IC:K.@92R WA-J.IT:G.A73L B.:-TO71WK: >@H:@LO75W00
text-trans-full-ketiv     WA-J.;71C:T.: MIN&HA-J.A73JIN WA-J.IC:K.@92R WA-J.IT:G.A73L B.:-TO71WK: *>HLH 
text-trans-plain          WJCT MN&HJJN WJCKR WJTGL BTWK >HLH 


# Edge features: mother

Let us do a few basic enquiries on an edge feature:
[mother](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/mother).

We count how many mothers nodes can have (it turns to be 0 or 1).
We walk through all nodes and per node we retrieve the mother nodes, and
we store the lengths (if non-zero) in a dictionary (`mother_len`).

We see that nodes have at most one mother.

We also count the inverse relationship: daughters.

In [39]:
motherLen = {}
daughterLen = {}
info('Counting edges')
for c in N():
    lms = E.mother.f(c) or []
    lds = E.mother.t(c) or []
    nms = len(lms)
    nds = len(lds)
    if nms: motherLen[c] = nms
    if nds: daughterLen[c] = nds
info('{} nodes have mothers'.format(len(motherLen)))
info('{} nodes have daughters'.format(len(daughterLen)))

motherCount = collections.Counter()
daughterCount = collections.Counter()

for (n, lm) in motherLen.items(): motherCount[lm] += 1
for (n, ld) in daughterLen.items(): daughterCount[ld] += 1

print('mothers', motherCount)
print('daughters', daughterCount)

    11s Counting edges
    14s 182159 nodes have mothers
    14s 144059 nodes have daughters
mothers Counter({1: 182159})
daughters Counter({1: 117926, 2: 17408, 3: 6272, 4: 1843, 5: 462, 6: 122, 7: 21, 8: 5})


# Export to Emdros MQL

[EMDROS](http://emdros.org), written by Ulrik Petersen.
is a text database system with the powerful *topographic* query language MQL.
The ideas are based on a model devised by Christ-Jan Doedens in
[Text Databases: One Database Model and Several Retrieval Languages](https://books.google.nl/books?id=9ggOBRz1dO4C).

Text-Fabric's model of slots, nodes and edges is a fairly straightforward translation of the models of Christ-Jan Doedens and Ulrik Petersen.

[SHEBANQ](https://shebanq.ancient-data.org) uses EMDROS to offer users to execute and save MQL queries against the Hebrew Text Database [BHSA](https://github.com/ETCBC/BHSA).

So it is kind of logical and convenient to be able to work with a Text-Fabric resource through MQL.

After the `Fabric(modules=...)` call, you can call `exportMQL()` in order to save all features of the
indicated modules into a big MQL dump, which can be imported by an EMDROS database.

In [40]:
TF.exportMQL(f'BHSA{VERSION}','~/Downloads')

  0.00s Checking features of dataset BHSA2017


   |     0.00s feature "book@am" => "book_am"
   |     0.00s feature "book@ar" => "book_ar"
   |     0.00s feature "book@bn" => "book_bn"
   |     0.00s feature "book@da" => "book_da"
   |     0.00s feature "book@de" => "book_de"
   |     0.00s feature "book@el" => "book_el"
   |     0.00s feature "book@en" => "book_en"
   |     0.00s feature "book@es" => "book_es"
   |     0.00s feature "book@fa" => "book_fa"
   |     0.00s feature "book@fr" => "book_fr"
   |     0.00s feature "book@he" => "book_he"
   |     0.00s feature "book@hi" => "book_hi"
   |     0.00s feature "book@id" => "book_id"
   |     0.00s feature "book@ja" => "book_ja"
   |     0.00s feature "book@ko" => "book_ko"
   |     0.00s feature "book@la" => "book_la"
   |     0.00s feature "book@nl" => "book_nl"
   |     0.00s feature "book@pa" => "book_pa"
   |     0.00s feature "book@pt" => "book_pt"
   |     0.00s feature "book@ru" => "book_ru"
   |     0.00s feature "book@sw" => "book_sw"
   |     0.00s feature "book@syc" 

   |     0.00s M code                 from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M det                  from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M dist                 from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M dist_unit            from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M distributional_parent from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M domain               from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M freq_occ             from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M function             from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M functional_parent    from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M g_nme                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M g_nme_utf8           from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M g_pfm                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M g_pfm_utf8

   |     0.00s feature "omap@2016-2017" => "omap_2016_2017"


   |     0.00s M otext@phono          from /Users/dirk/github/etcbc/phono/tf/2017
   |     0.00s M pargr                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M pdp                  from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M pfm                  from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M prs                  from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M prs_gn               from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M prs_nu               from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M prs_ps               from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M ps                   from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M rank_lex             from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M rank_occ             from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M rela                 from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.00s M root      

	g_nme          :  108 values, 108 not a name, e.g. «»
	g_nme_utf8     :  106 values, 106 not a name, e.g. «»
	g_pfm          :   87 values, 87 not a name, e.g. «»
	g_pfm_utf8     :   86 values, 86 not a name, e.g. «»
	g_prs          :  127 values, 127 not a name, e.g. «»
	g_prs_utf8     :  126 values, 126 not a name, e.g. «»
	g_uvf          :   19 values, 19 not a name, e.g. «»
	g_uvf_utf8     :   17 values, 17 not a name, e.g. «»
	g_vbe          :  101 values, 101 not a name, e.g. «»
	g_vbe_utf8     :   97 values, 97 not a name, e.g. «»
	g_vbs          :   66 values, 66 not a name, e.g. «»
	g_vbs_utf8     :   65 values, 65 not a name, e.g. «»
	instruction    :   35 values, 20 not a name, e.g. «.#»
	nametype       :    9 values, 4 not a name, e.g. «gens,topo»
	nme            :   20 values, 7 not a name, e.g. «»
	pfm            :   11 values, 4 not a name, e.g. «»
	phono_trailer  :    4 values, 4 not a name, e.g. «»
	prs            :   22 values, 4 not a name, e.g. «H=»
	qere_trailer  

Now you have a file `~/Downloads/BHSA2017.mql` of ca. 600 MB.
You can import it into an Emdros database by saying:

    cd ~/Downloads
    rm BHSA2017
    mql -b 3 < BHSA2017.mql
    
It will take some time!
You'll see progress messages like

```
dirk:~/Downloads > mql -b 3 < BHSA2017.mql
Dropping indices on word_objects...!
+------------------------+
| object_count : integer |
+------------------------+
| 50000                  |
+------------------------+
+------------------------+
| object_count : integer |
+------------------------+
| 50000                  |
+------------------------+
```

The result is an sqlite3 database `BHSA2017` in the same directory (168 MB).
You can run a query against it by creating a text file test.mql with this contents:

    select all objects where
    [lex gloss ~ 'create'
        [word FOCUS]
    ]
    
And then say

    mql -b 3 -d BHSA2017 test.mql
    
You will see raw query results: all word occurrences that belong to lexemes with `create` in their gloss.

It looks like this:

```
 //  <  < [ lex 1437405 { 3, 381, 535, 545, 550, 724, 736, 2126, 2137, 2148, 2740, 47988, 80620, 95897, 213379, 225969, 226014, 226387, 226589, 226941, 227051, 227178, 227979, 227984, 228010, 228065, 228171, 228186, 228896, 231015, 231028, 231865, 234285, 234310, 234313, 251053, 275157, 278572, 278597, 296496, 309970, 318972, 325849, 326167, 327907, 328521, 335786, 363235 } false  //  <  < [ word 3 { 3 } true  //  <  > 
 ]
 > 
 < [ word 381 { 381 } true  //  <  > 
 ]
 > 
 < [ word 535 { 535 } true  //  <  > 
 ]
 > 
 < [ word 545 { 545 } true  //  <  > 
 ]
 > 
 < [ word 550 { 550 } true  //  <  > 
 ]
 > 
 < [ word 724 { 724 } true  //  <  > 
 ]
 > 
 < [ word 736 { 736 } true  //  <  > 
 ]
 > 
 < [ word 2126 { 2126 } true  //  <  > 
 ]
 > 
 < [ word 2137 { 2137 } true  //  <  > 
 ]
 > 
 < [ word 2148 { 2148 } true  //  <  > 
 ]
 > 
 < [ word 2740 { 2740 } true  //  <  > 
 ]
 > 
 < [ word 47988 { 47988 } true  //  <  > 
 ]
 > 
 < [ word 80620 { 80620 } true  //  <  > 
 ]
 > 
 < [ word 95897 { 95897 } true  //  <  > 
 ]
 > 
 < [ word 213379 { 213379 } true  //  <  > 
 ]
 > 
 < [ word 225969 { 225969 } true  //  <  > 
 ]
 > 
 < [ word 226014 { 226014 } true  //  <  > 
 ]
 > 
 < [ word 226387 { 226387 } true  //  <  > 
 ]
 > 
 < [ word 226589 { 226589 } true  //  <  > 
 ]
 > 
 < [ word 226941 { 226941 } true  //  <  > 
 ]
 > 
 < [ word 227051 { 227051 } true  //  <  > 
 ]
 > 
 < [ word 227178 { 227178 } true  //  <  > 
 ]
 > 
 < [ word 227979 { 227979 } true  //  <  > 
 ]
 > 
 < [ word 227984 { 227984 } true  //  <  > 
 ]
 > 
 < [ word 228010 { 228010 } true  //  <  > 
 ]
 > 
 < [ word 228065 { 228065 } true  //  <  > 
 ]
 > 
 < [ word 228171 { 228171 } true  //  <  > 
 ]
 > 
 < [ word 228186 { 228186 } true  //  <  > 
 ]
 > 
 < [ word 228896 { 228896 } true  //  <  > 
 ]
 > 
 < [ word 231015 { 231015 } true  //  <  > 
 ]
 > 
 < [ word 231028 { 231028 } true  //  <  > 
 ]
 > 
 < [ word 231865 { 231865 } true  //  <  > 
 ]
 > 
 < [ word 234285 { 234285 } true  //  <  > 
 ]
 > 
 < [ word 234310 { 234310 } true  //  <  > 
 ]
 > 
 < [ word 234313 { 234313 } true  //  <  > 
 ]
 > 
 < [ word 251053 { 251053 } true  //  <  > 
 ]
 > 
 < [ word 275157 { 275157 } true  //  <  > 
 ]
 > 
 < [ word 278572 { 278572 } true  //  <  > 
 ]
 > 
 < [ word 278597 { 278597 } true  //  <  > 
 ]
 > 
 < [ word 296496 { 296496 } true  //  <  > 
 ]
 > 
 < [ word 309970 { 309970 } true  //  <  > 
 ]
 > 
 < [ word 318972 { 318972 } true  //  <  > 
 ]
 > 
 < [ word 325849 { 325849 } true  //  <  > 
 ]
 > 
 < [ word 326167 { 326167 } true  //  <  > 
 ]
 > 
 < [ word 327907 { 327907 } true  //  <  > 
 ]
 > 
 < [ word 328521 { 328521 } true  //  <  > 
 ]
 > 
 < [ word 335786 { 335786 } true  //  <  > 
 ]
 > 
 < [ word 363235 { 363235 } true  //  <  > 
 ]
 > 
 > 
 ]
 > 
 < [ lex 1437665 { 1682, 6579, 6625, 111521, 334458, 349029 } false  //  <  < [ word 1682 { 1682 } true  //  <  > 
 ]
 > 
 < [ word 6579 { 6579 } true  //  <  > 
 ]
 > 
 < [ word 6625 { 6625 } true  //  <  > 
 ]
 > 
 < [ word 111521 { 111521 } true  //  <  > 
 ]
 > 
 < [ word 334458 { 334458 } true  //  <  > 
 ]
 > 
 < [ word 349029 { 349029 } true  //  <  > 
 ]
 > 
 > 
 ]
 > 
 < [ lex 1440298 { 80619 } false  //  <  < [ word 80619 { 80619 } true  //  <  > 
 ]
 > 
 > 
 ]
 > 
 > 

```
     
It is not very pretty, and probably you should use a more visual Emdros tool to run those queries.
But, while we're at it, observe how the word object ids coincide with their monad numbers.

And let's look up lexeme 1437665 here in TF, together with its 6 occurrences.

In [41]:
lexNode = 1437665
print('{} {} "{}"'.format(
    F.lex.v(lexNode),
    F.voc_lex_utf8.v(lexNode),
    F.gloss.v(lexNode),
))
for wordNode in L.d(lexNode, otype='word'):
    verseNode = L.u(wordNode, otype='verse')[0]
    print('{} {}\n\t{}'.format(
        '{} {}:{}'.format(*T.sectionFromNode(wordNode)),
        F.g_word.v(wordNode),
        T.text(L.d(verseNode, otype='word')),
    ))

QNH=[ קנה "create"
Genesis 4:1 Q@NI71JTIJ
	וְהָ֣אָדָ֔ם יָדַ֖ע אֶת־חַוָּ֣ה אִשְׁתֹּ֑ו וַתַּ֨הַר֙ וַתֵּ֣לֶד אֶת־קַ֔יִן וַתֹּ֕אמֶר קָנִ֥יתִי אִ֖ישׁ אֶת־יְהוָֽה׃ 
Genesis 14:19 QON;73H
	וַֽיְבָרְכֵ֖הוּ וַיֹּאמַ֑ר בָּר֤וּךְ אַבְרָם֙ לְאֵ֣ל עֶלְיֹ֔ון קֹנֵ֖ה שָׁמַ֥יִם וָאָֽרֶץ׃ 
Genesis 14:22 QON;73H
	וַיֹּ֥אמֶר אַבְרָ֖ם אֶל־מֶ֣לֶךְ סְדֹ֑ם הֲרִימֹ֨תִי יָדִ֤י אֶל־יְהוָה֙ אֵ֣ל עֶלְיֹ֔ון קֹנֵ֖ה שָׁמַ֥יִם וָאָֽרֶץ׃ 
Deuteronomy 32:6 Q.@NE80K@
	הֲ־לַיְהוָה֙ תִּגְמְלוּ־זֹ֔את עַ֥ם נָבָ֖ל וְלֹ֣א חָכָ֑ם הֲלֹוא־הוּא֙ אָבִ֣יךָ קָּנֶ֔ךָ ה֥וּא עָֽשְׂךָ֖ וַֽיְכֹנְנֶֽךָ׃ 
Psalms 139:13 Q@NI74JT@
	כִּֽי־אַ֭תָּה קָנִ֣יתָ כִלְיֹתָ֑י תְּ֝סֻכֵּ֗נִי בְּבֶ֣טֶן אִמִּֽי׃ 
Proverbs 8:22 13Q@N@NIJ
	יְֽהוָ֗ה קָ֭נָנִי רֵאשִׁ֣ית דַּרְכֹּ֑ו קֶ֖דֶם מִפְעָלָ֣יו מֵאָֽז׃ 


# Clean caches

Text-Fabric precomputes data for you, so that it can be loaded faster.
If the original data is updated, Text-Fabric detects it, and will recompute that data.

But there are cases, when the algorithms of Text-Fabric have changed, without any changes in the data, that you might
want to clear the cache of precomputed results.

There are two ways to do that:

* Locate the `.tf` directory of your dataset, and remove all `.tfx` files in it.
  This might be a bit awkward to do, because the `.tf` directory is hidden on Unix-like systems.
* Call `TF.clearCache()`, which does exactly the same.

It is not handy to execute the following cell all the time, that's why I have commented it out.
So if you really want to clear the cache, remove the comment sign below.

In [42]:
# TF.clearCache()